# Nearest Neighbor Recommender: People behind Movies

In [1146]:
import os
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 600)
from sklearn import preprocessing
from sklearn.neighbors import NearestNeighbors

In [1147]:
data_path = os.path.join(os.getcwd(), 'data')
movies = pd.read_csv(os.path.join(data_path, 'movies_preprocessed.csv'))
movies = movies.fillna("")

### Remove unimportant features

In [1148]:
features = ['id','title','star_1','star_2', 'star_3', 'writer', 'producer', 'director']
movies = movies[features]

### Remove unimportant actors

In [1149]:
def remove_unimportant_items(movie_dataset, column, file, top):
    popular_items = pd.read_json(os.path.join(data_path, file))[:top]
        
    #set all non popular actors null
    for idx, movie in movie_dataset.iterrows():
        if(not any(popular_items['name'] == movie[column])):
            movie_dataset.at[idx, column] = ""
        else:
            movie_dataset.at[idx, column] = [movie_dataset.at[idx, column]]
            
    return movie_dataset

In [1150]:
#transform actors
movies = remove_unimportant_items(movies, 'star_1', 'popular_actors.json', 500)
movies = remove_unimportant_items(movies, 'star_2', 'popular_actors.json', 300)
movies = remove_unimportant_items(movies, 'star_3', 'popular_actors.json', 100)

In [1151]:
#transform writers
movies = remove_unimportant_items(movies, 'writer', 'top_writer.json', 14)

In [1152]:
#transform directors
movies = remove_unimportant_items(movies, 'director', 'top_directors.json', 20)

In [1153]:
#transform producers
movies = remove_unimportant_items(movies, 'producer', 'top_producers.json', 12)

### Transforming features

In [1154]:
def feature_one_hot(movie_dataset, column): 
    'Returns the given movie dataset with a transformed to one-hot encoded genre column.'

    # MultiLabelBinarizer is a very fast solution for one hot encoding on large dataframes
    mlb = MultiLabelBinarizer()
        
    one_hot_genre = pd.DataFrame(mlb.fit_transform(movie_dataset[column]),
                                 columns= [column + ' ' + str(class_) for class_ in mlb.classes_], 
                                 index=movie_dataset[column].index)

    movie_dataset = pd.concat([movie_dataset, one_hot_genre], sort=False, axis=1)
    movie_dataset.drop(column, axis=1)
    
    return movie_dataset

In [1155]:
#Actors
movies = feature_one_hot(movies, 'star_1')
movies = feature_one_hot(movies, 'star_2')
movies = feature_one_hot(movies, 'star_3')
del movies['star_1']
del movies['star_2']
del movies['star_3']

In [1156]:
#Writers
movies = feature_one_hot(movies, 'writer')
del movies['writer']

In [1157]:
#Directors
movies = feature_one_hot(movies, 'director')
del movies['director']

In [1158]:
#Producers
movies = feature_one_hot(movies, 'producer')
del movies['producer']

## NN_Recommender

In [1159]:
class NNRecommender():
    
    def __init__(self, data, k=6, metric='minkowski', leaf_size=30):
        self.data = data.drop('title', axis=1)
        self.nn = NearestNeighbors(n_neighbors=k, metric=metric, leaf_size=leaf_size)
        self.nn.fit(self.data.drop('id', axis=1))
    
    def _indices_to_movie_id(self, indices):
        movie_ids = []
        for index in indices:
            movie_ids.append(self.data.iloc[index]['id'].values)
        return movie_ids[0][1:]
    
    def recommend(self, movie_id, return_distance=False):
        movie = self.data[self.data['id']==movie_id].drop('id', axis=1).values
        distances, indices = self.nn.kneighbors(movie)
        recommendations = self._indices_to_movie_id(indices)
        if return_distance:
            return recommendations, distances
        else:
            return recommendations

In [1160]:
def translate_id_to_title(ids, data):
    titles = []
    for i in ids:
        titles.append(data[data['id'] == i]['title'].values[0])
    return titles

In [1161]:
nnr = NNRecommender(movies,
                    k=6, 
                    metric='minkowski', 
                    leaf_size=30)

## Only Actors Top 100

In [887]:
test_movies = [13, 1726, 680, 665, 583, 103, 128]
for movie_id in test_movies:
    print('Movie: ', translate_id_to_title([movie_id], movies))
    recommendations = nnr.recommend(movie_id=movie_id)
    titles = translate_id_to_title(recommendations, movies)
    print('Recommendations:\n{}'.format(titles))
    print()

Movie:  ['Forrest Gump']
Recommendations:
["The 'Burbs", 'Forrest Gump', 'Toy Story', 'Apollo 13', 'Sleepless in Seattle']

Movie:  ['Iron Man']
Recommendations:
['The Singing Detective', 'Heart and Souls', 'Restoration', 'Two Girls and a Guy', '1969']

Movie:  ['Pulp Fiction']
Recommendations:
['Changing Lanes', 'Jackie Brown', 'Pulp Fiction', 'The Long Kiss Goodnight', 'Unbreakable']

Movie:  ['Ben-Hur']
Recommendations:
['A Cruel Romance', 'The Jungle Book', "Things to Do in Denver When You're Dead", 'Jury Duty', 'Train of Life']

Movie:  ['Life of Brian']
Recommendations:
['A Cruel Romance', 'The Jungle Book', "Things to Do in Denver When You're Dead", 'Jury Duty', 'Train of Life']

Movie:  ['Taxi Driver']
Recommendations:
['A Cruel Romance', 'The Jungle Book', "Things to Do in Denver When You're Dead", 'Jury Duty', 'Train of Life']

Movie:  ['Princess Mononoke']
Recommendations:
['A Cruel Romance', 'The Jungle Book', "Things to Do in Denver When You're Dead", 'Jury Duty', 'Train o

## Only Actors Top 200

In [859]:
test_movies = [13, 1726, 680, 665, 583, 103, 128]
for movie_id in test_movies:
    print('Movie: ', translate_id_to_title([movie_id], movies))
    recommendations = nnr.recommend(movie_id=movie_id)
    titles = translate_id_to_title(recommendations, movies)
    print('Recommendations:\n{}'.format(titles))
    print()

Movie:  ['Forrest Gump']
Recommendations:
["The 'Burbs", 'Forrest Gump', 'Toy Story', 'Apollo 13', 'Sleepless in Seattle']

Movie:  ['Iron Man']
Recommendations:
['Chaplin', 'Heart and Souls', 'Two Girls and a Guy', '1969', 'The Singing Detective']

Movie:  ['Pulp Fiction']
Recommendations:
['xXx: State of the Union', 'Jackie Brown', 'Pulp Fiction', 'The Long Kiss Goodnight', 'Twisted']

Movie:  ['Ben-Hur']
Recommendations:
['Testament of Orpheus', 'Dreaming of Joseph Lees', "Things to Do in Denver When You're Dead", 'Kids', 'Turbo: A Power Rangers Movie']

Movie:  ['Life of Brian']
Recommendations:
['Testament of Orpheus', 'Dreaming of Joseph Lees', "Things to Do in Denver When You're Dead", 'Kids', 'Turbo: A Power Rangers Movie']

Movie:  ['Taxi Driver']
Recommendations:
['Testament of Orpheus', 'Dreaming of Joseph Lees', "Things to Do in Denver When You're Dead", 'Kids', 'Turbo: A Power Rangers Movie']

Movie:  ['Princess Mononoke']
Recommendations:
['Testament of Orpheus', 'Dreamin

## Top 100 Actors and Top 14 Writer

In [964]:
test_movies = [13, 1726, 680, 665, 583, 103, 128]
for movie_id in test_movies:
    print('Movie: ', translate_id_to_title([movie_id], movies))
    recommendations = nnr.recommend(movie_id=movie_id)
    titles = translate_id_to_title(recommendations, movies)
    print('Recommendations:\n{}'.format(titles))
    print()

Movie:  ['Forrest Gump']
Recommendations:
["The 'Burbs", 'Forrest Gump', 'Toy Story', 'Apollo 13', 'Philadelphia']

Movie:  ['Iron Man']
Recommendations:
['The Singing Detective', 'Heart and Souls', 'Restoration', 'Two Girls and a Guy', '1969']

Movie:  ['Pulp Fiction']
Recommendations:
['Changing Lanes', 'Jackie Brown', 'Pulp Fiction', 'The Long Kiss Goodnight', 'Unbreakable']

Movie:  ['Ben-Hur']
Recommendations:
['True Confessions', 'I Like It Like That', "Things to Do in Denver When You're Dead", 'Jury Duty', 'The Guardian']

Movie:  ['Life of Brian']
Recommendations:
['True Confessions', 'I Like It Like That', "Things to Do in Denver When You're Dead", 'Jury Duty', 'The Guardian']

Movie:  ['Taxi Driver']
Recommendations:
['True Confessions', 'I Like It Like That', "Things to Do in Denver When You're Dead", 'Jury Duty', 'The Guardian']

Movie:  ['Princess Mononoke']
Recommendations:
['True Confessions', 'I Like It Like That', "Things to Do in Denver When You're Dead", 'Jury Duty',

## Top 500, 300, 100 Actors, Top 14 Writer

In [981]:
test_movies = [13, 1726, 680, 665, 583, 103, 128]
for movie_id in test_movies:
    print('Movie: ', translate_id_to_title([movie_id], movies))
    recommendations = nnr.recommend(movie_id=movie_id)
    titles = translate_id_to_title(recommendations, movies)
    print('Recommendations:\n{}'.format(titles))
    print()

Movie:  ['Forrest Gump']
Recommendations:
["The 'Burbs", 'Forrest Gump', 'Toy Story', 'Apollo 13', 'Sleepless in Seattle']

Movie:  ['Iron Man']
Recommendations:
['The Singing Detective', 'Black and White', 'Two Girls and a Guy', 'Heart and Souls', '1969']

Movie:  ['Pulp Fiction']
Recommendations:
['Zambezia', "National Lampoon's Loaded Weapon 1", 'Pulp Fiction', 'The Long Kiss Goodnight', '1408']

Movie:  ['Ben-Hur']
Recommendations:
['Earthquake', 'Soylent Green', 'Ben-Hur', 'Touch of Evil', 'Planet of the Apes']

Movie:  ['Life of Brian']
Recommendations:
['Personal Velocity', 'Rawhead Rex', "Things to Do in Denver When You're Dead", 'Kids', 'Selena']

Movie:  ['Taxi Driver']
Recommendations:
['Once Upon a Time in America', 'A Bronx Tale', 'Taxi Driver', "Mary Shelley's Frankenstein", 'GoodFellas']

Movie:  ['Princess Mononoke']
Recommendations:
['Personal Velocity', 'Rawhead Rex', "Things to Do in Denver When You're Dead", 'Kids', 'Selena']



## Top 500, 300, 100 Actors, Top 14 Writer, Top 20 Directors

In [1067]:
test_movies = [13, 1726, 680, 665, 583, 103, 128]
for movie_id in test_movies:
    print('Movie: ', translate_id_to_title([movie_id], movies))
    recommendations = nnr.recommend(movie_id=movie_id)
    titles = translate_id_to_title(recommendations, movies)
    print('Recommendations:\n{}'.format(titles))
    print()

Movie:  ['Forrest Gump']
Recommendations:
["The 'Burbs", 'Forrest Gump', 'Toy Story', 'Apollo 13', 'Sleepless in Seattle']

Movie:  ['Iron Man']
Recommendations:
['The Singing Detective', 'Black and White', 'Two Girls and a Guy', 'Heart and Souls', '1969']

Movie:  ['Pulp Fiction']
Recommendations:
['Death Proof', 'Jackie Brown', "National Lampoon's Loaded Weapon 1", 'The Long Kiss Goodnight', 'Twisted']

Movie:  ['Ben-Hur']
Recommendations:
['Earthquake', 'Soylent Green', 'Ben-Hur', 'Touch of Evil', 'Planet of the Apes']

Movie:  ['Life of Brian']
Recommendations:
['Personal Velocity', 'Rawhead Rex', "Things to Do in Denver When You're Dead", 'Kids', 'Selena']

Movie:  ['Taxi Driver']
Recommendations:
['The King of Comedy', 'Taxi Driver', 'Raging Bull', 'A Bronx Tale', 'The Age of Innocence']

Movie:  ['Princess Mononoke']
Recommendations:
['Personal Velocity', 'Rawhead Rex', "Things to Do in Denver When You're Dead", 'Kids', 'Selena']



## Top 500, 300, 100 Actors, Top 14 Writer, Top 20 Directors, 12 Producers

In [1179]:
test_movies = [13, 1726, 680, 665, 583, 103, 128, 15121, 1984, 562, 176]
for movie_id in test_movies:
    print('Movie: ', translate_id_to_title([movie_id], movies))
    recommendations = nnr.recommend(movie_id=movie_id)
    titles = translate_id_to_title(recommendations, movies)
    print('Recommendations:\n{}'.format(titles))
    print()

Movie:  ['Forrest Gump']
Recommendations:
["The 'Burbs", 'Forrest Gump', 'Toy Story', 'Apollo 13', 'Sleepless in Seattle']

Movie:  ['Iron Man']
Recommendations:
['The Singing Detective', 'Black and White', 'Two Girls and a Guy', 'Heart and Souls', '1969']

Movie:  ['Pulp Fiction']
Recommendations:
['Death Proof', 'Jackie Brown', "National Lampoon's Loaded Weapon 1", 'The Long Kiss Goodnight', 'Twisted']

Movie:  ['Ben-Hur']
Recommendations:
['Earthquake', 'Soylent Green', 'Ben-Hur', 'Touch of Evil', 'Planet of the Apes']

Movie:  ['Life of Brian']
Recommendations:
['Personal Velocity', 'Rawhead Rex', "Things to Do in Denver When You're Dead", 'Kids', 'Selena']

Movie:  ['Taxi Driver']
Recommendations:
['The King of Comedy', 'Taxi Driver', 'Raging Bull', 'A Bronx Tale', 'The Age of Innocence']

Movie:  ['Princess Mononoke']
Recommendations:
['Personal Velocity', 'Rawhead Rex', "Things to Do in Denver When You're Dead", 'Kids', 'Selena']

Movie:  ['The Sound of Music']
Recommendations:


In [1180]:
#Print all column names with values != 0
movie_name = "Saw"
for mov in movies[movies['title']== movie_name]:
    if(movies[movies['title']==movie_name][mov].item() != 0):
        print(mov)


ValueError: can only convert an array of size 1 to a Python scalar

In [1011]:
movies[movies['title']== 'Ben-Hur']['producer']

1258        Sam Zimbalist
26591    Duncan Henderson
Name: producer, dtype: object

In [1085]:
#Print all column names with values != 0
movie_name = "Black and White"
for mov in movies[movies['title']== movie_name]:
    if(movies[movies['title']==movie_name][mov].item() != 0):
        print(mov)


ValueError: can only convert an array of size 1 to a Python scalar

In [1178]:
movies[movies['title'] == 'Saw']

,id,title,star_1 50 Cent,star_1 Aaron Paul,star_1 Adam Baldwin,star_1 Adam Sandler,star_1 Aishwarya Rai Bachchan,star_1 Al Pacino,star_1 Alain Delon,star_1 Alec Baldwin,star_1 Alexa PenaVega,star_1 Alexandra Daddario,star_1 Ali Larter,star_1 Alice Eve,star_1 Alycia Debnam-Carey,star_1 Amanda Seyfried,star_1 Amber Heard,star_1 Amy Adams,star_1 Amy Poehler,star_1 Amy Smart,star_1 Angelina Jolie,star_1 Angie Harmon,star_1 Anjelica Huston,star_1 Ann-Margret,star_1 Anna Faris,star_1 Anna Raadsveld,star_1 Anne Bancroft,star_1 Anne Hathaway,star_1 Anthony Hopkins,star_1 Anton Yelchin,star_1 Arielle Kebbel,star_1 Armand Assante,star_1 Arnold Schwarzenegger,star_1 Ashley Benson,star_1 Ashley Greene,star_1 Ashton Kutcher,star_1 Beau Bridges,star_1 Ben Affleck,star_1 Ben Chaplin,star_1 Ben Stiller,star_1 Bill Murray,star_1 Bill Paxton,star_1 Brad Pitt,star_1 Bradley Cooper,star_1 Brendan Gleeson,star_1 Bridget Fonda,star_1 Brigitte Nielsen,star_1 Bruce Willis,star_1 Bryan Cranston,star_1 C. Thomas Howell,star_1 Cameron Diaz,star_1 Carice van Houten,star_1 Carla Gugino,star_1 Carmen Electra,star_1 Carrie-Anne Moss,star_1 Cate Blanchett,star_1 Catherine Zeta-Jones,star_1 Channing Tatum,star_1 Charlize Theron,star_1 Charlotte Gainsbourg,star_1 Charlotte Rampling,star_1 Charlton Heston,star_1 Chris Evans,star_1 Chris Hemsworth,star_1 Chris Pine,star_1 Chris Pratt,star_1 Chris Rock,star_1 Christian Bale,star_1 Christina Hendricks,star_1 Christopher Lambert,star_1 Christopher Lee,star_1 Christopher Lloyd,star_1 Christopher Walken,star_1 Claire Danes,star_1 Claudia Cardinale,star_1 Claudia Koll,star_1 Cliff Curtis,star_1 Clint Eastwood,star_1 Clive Owen,star_1 Colin Farrell,star_1 Colin Firth,star_1 Colm Meaney,star_1 Connie Nielsen,star_1 Dabney Coleman,star_1 Dakota Blue Richards,star_1 Dakota Fanning,star_1 Dan Duryea,star_1 Dana Ashbrook,star_1 Daniel Craig,star_1 Daniel Radcliffe,star_1 Danielle Panabaker,star_1 Danny Glover,star_1 Danny Trejo,star_1 Dave Bautista,star_1 David Hemmings,star_1 Demi Moore,star_1 Denise Richards,star_1 Dennis Chan,star_1 Dennis Hopper,star_1 Denzel Washington,star_1 Diane Lane,star_1 Dolph Lundgren,star_1 Dominic Cooper,star_1 Dominic Purcell,star_1 Donald Sutherland,star_1 Donnie Yen,star_1 Dougray Scott,star_1 Drew Barrymore,star_1 Dwayne Johnson,star_1 Ed Asner,star_1 Ed Harris,star_1 Eddie Murphy,star_1 Eddie Redmayne,star_1 Edward Furlong,star_1 Edward Norton,star_1 Edwige Fenech,star_1 Elijah Wood,star_1 Elisabeth Shue,star_1 Elisha Cuthbert,star_1 Elizabeth Banks,star_1 Ellen Page,star_1 Emile Hirsch,star_1 Emilia Clarke,star_1 Emily Blunt,star_1 Emily Watson,star_1 Emma Roberts,star_1 Emma Stone,star_1 Emma Watson,star_1 Emmanuelle Chriqui,star_1 Emmy Rossum,star_1 Eric Bana,star_1 Eric Roberts,star_1 Ernest Borgnine,star_1 Ethan Hawke,star_1 Eva Green,star_1 Eva Mendes,star_1 Ewan McGregor,star_1 Famke Janssen,star_1 Fan Bingbing,star_1 Felicity Jones,star_1 Forest Whitaker,star_1 Garrett Hedlund,star_1 Gaspard Ulliel,star_1 Geoffrey Rush,star_1 George Clooney,star_1 George Takei,star_1 Gerard Butler,star_1 Gina Gershon,star_1 Greg Kinnear,star_1 Gwyneth Paltrow,star_1 Harrison Ford,star_1 Harvey Keitel,star_1 Hayden Christensen,star_1 Hayley Atwell,star_1 Heather Graham,star_1 Helen Hunt,star_1 Helen Mirren,star_1 Henry Winkler,star_1 Hikari Mitsushima,star_1 Hugh Grant,star_1 Hugh Jackman,star_1 Ian McKellen,star_1 Ian Somerhalder,star_1 Ice Cube,star_1 Ida Lupino,star_1 Ioan Gruffudd,star_1 Izabella Miko,star_1 Jack Black,star_1 Jack Nicholson,star_1 Jackie Chan,star_1 Jaime Pressly,star_1 Jake Gyllenhaal,star_1 Jake Johnson,star_1 Jake McDorman,star_1 James Cameron,star_1 James Coburn,star_1 James Gandolfini,star_1 James Marsden,star_1 James McAvoy,star_1 James Purefoy,star_1 James Spader,star_1 Jamie Foxx,star_1 Jamie Lee Curtis,star_1 Jason Clarke,star_1 Jason Segel,star_1 Jason Statham,star_1 Jean-Claude Van Damme,star_1 Jennifer Aniston,star_1 Jennifer Connelly,star_1 Jennifer Lawrence,star_1

In [1121]:
movies.head()

,id,title,star_1,star_2,star_3,writer,producer,director,actors
0,862,Toy Story,[Tom Hanks],,,,Bonnie Arnold,,Tom Hanks
1,8844,Jumanji,[Robin Williams],,[Kirsten Dunst],,Scott Kroopf,,[Robin Williams]
2,15602,Grumpier Old Men,,,,,,,
3,31357,Waiting to Exhale,,,,,Ronald Bass,,
4,11862,Father of the Bride Part II,[Steve Martin],,,,Nancy Meyers,,[Steve Martin]


In [1117]:
movies['actors']

KeyError: 'actors'